In [1]:
# https://nlp.johnsnowlabs.com/docs/en/annotators#languagedetectordl

# maven - com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.3
# pypi - spark-nlp, vaderSentiment, boto3==1.9.157

# example on sentiment analysis
  # https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/SentimentDL_train_multiclass_sentiment_classifier.ipynb
# https://nlp.johnsnowlabs.com/docs/en/pipelines#multi-language

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark import SparkFiles
from pyspark.ml.feature import (
    StringIndexer,
    HashingTF, 
    IDF
)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes

In [3]:
# get or create Spark session

app_name = "news_analysis"

spark = SparkSession.builder.appName(app_name) \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.3") \
    .getOrCreate()

In [4]:
file_uri = "/mnt/mnt_s3/TechTrainData_1.csv"

trainDataset = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(file_uri)
display(trainDataset)

_c0,index,category,headline,authors,link,short_description,date,short_description_new,label
59991,62030,TECH,Report: White House Will Not Publicly Back Encryption Legislation,"Mark Hosenball and Dustin Volz, Reuters",https://www.huffingtonpost.com/entry/report-white-house-will-not-publicly-back-encryption-legislation_us_5705b08fe4b0b90ac2713d65,administration remains deeply divided issue sources said,2016-04-07,administration remains deeply divided issue sources said,Neutral
175823,198288,TECH,"With New Funding and Fellows, Code for America Looks to Scale Civic Innovation Across the Country","Alexander Howard, Contributor",null,null,null,null,null
Senior Editor for Technology and Society,"The Huffington Post.""",https://www.huffingtonpost.com/entry/code-for-america_us_5bb34cc4e4b0fa920b95e6df,given economic pressures challenges confront hundreds millions americans need killer civic app greater,2012-02-25,given economic pressures challenges confront hundreds millions americans need killer civic app greater,Negative,null,null,null
46674,47896,TECH,Samsung Galaxy Note 7 Recalled Over Battery Fires,"Ankur Banerjee, Reuters",https://www.huffingtonpost.com/entry/us-consumer-product-safety-commission-reportedly-plans-recall-of-samsung-galaxy-note-7_us_57dafcffe4b08cb1409476ec,agency said owners smartphones stop devices turn threat battery,2016-09-15,agency said owners smartphones stop devices turn threat battery,Negative
15783,16008,TECH,The 7 Problems With Our Phones,"The School Of Life, ContributorThe School Of Life",https://www.huffingtonpost.com/entry/the-7-problems-with-our-phones_us_59c13102e4b0f96732cbc938,addicted phones rely extent recruit harmful project selfavoidance,2017-09-21,addicted phones rely extent recruit harmful project selfavoidance,Neutral
61052,63129,TECH,Interactive Maps Show You What Cities Sound Like,Casey Williams,https://www.huffingtonpost.com/entry/chatty-maps-what-cities-sound-like_us_56f542dbe4b0a3721819b3c2,city sound,2016-03-25,city sound,Neutral
126804,148394,TECH,New iPhone To Be Unveiled By Apple September 10th: Report,"Reuters, Reuters",https://www.huffingtonpost.com/entry/new-iphone-apple_us_5bb368c6e4b0fa920b97ffbc,reuters apple worlds largest technology company expected present redesigned iphone september,2013-08-11,reuters apple worlds largest technology company expected present redesigned iphone september,Neutral
153436,175478,TECH,Google Event Cancelled Due To Hurricane Sandy,null,https://www.huffingtonpost.com/entry/google-event-cancelled-hurricane-sandy_us_5bb35abae4b0fa920b96e68f,meantime track hurricane sandy online handy twitter facebook page rundowns,2012-10-27,meantime track hurricane sandy online handy twitter facebook page rundowns,Neutral
512,519,TECH,Facebook Removed More Than 500 Million Fake Accounts In First Quarter Of 2018,Willa Frej,https://www.huffingtonpost.com/entry/facebook-removed-fake-accounts_us_5afbeed1e4b06a3fb50c0458,facebooks community standards enforcement report shows company trying open executive said,2018-05-16,facebooks community standards enforcement report shows company trying open executive said,Neutral
10868,11012,TECH,Elon Musk's Tweet Gives Creepy Insight Into Future Of Humanoid Robots,Doha Madani,https://www.huffingtonpost.com/entry/elon-musk-artificial-intelligence_us_5a1b3e02e4b0cee6c0508746,years bot fast youll need strobe light sweet dreams,2017-11-26,years bot fast youll need strobe light sweet dreams,Positive


In [5]:
file_uri = "/mnt/mnt_s3/TechnologyTestData_2.csv"

testDataset = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(file_uri)
display(testDataset)

_c0,index,category,headline,authors,link,short_description,date,short_description_new,label
136,137,TECH,Facebook Accused Of Reading Texts And Accessing Microphones In Lawsuit,Sara Boboltz,https://www.huffingtonpost.com/entry/facebook-allegations-reading-texts-accessing-microphones_us_5b06c226e4b07c4ea105b154,failed startup suggested facebook mounted described mass surveillance,2018-05-24,failed startup suggested facebook mounted described mass surveillance,Negative
137,138,TECH,Self-Driving Uber In Fatal Accident Had 6 Seconds To React Before Crash,Ryan Grenoble,https://www.huffingtonpost.com/entry/self-driving-uber-ntsb-report_us_5b06df0be4b05f0fc845e16a,ntsb published preliminary report incident thursday,2018-05-24,ntsb published preliminary report incident thursday,Neutral
270,272,TECH,Instagram Is Adding A 'Mute' Button For The Sake Of Your Sanity,Sara Boboltz,https://www.huffingtonpost.com/entry/instagram-is-introducing-a-mute-button-for-your-sanity_us_5b042c65e4b003dc7e469c3c,know use,2018-05-22,use,Neutral
369,373,TECH,Cambridge Analytica Files For Chapter 7 Bankruptcy After Facebook Privacy Row,null,https://www.huffingtonpost.com/entry/cambridge-analytica-bankruptcy_us_5afe7bd1e4b0463cdba0601a,firm center facebook data hack decision suffering sharp drop business,2018-05-18,firm center facebook data hack decision suffering sharp drop business,Negative
493,499,TECH,Why Isn't There A Birth Control Emoji?,"Nicole Lee, Engadget",https://www.huffingtonpost.com/entry/why-isnt-there-a-birth-control-emoji_us_5afc5e5ee4b0779345d51dc6,women decided time,2018-05-16,women decided time,Neutral
512,519,TECH,Facebook Removed More Than 500 Million Fake Accounts In First Quarter Of 2018,Willa Frej,https://www.huffingtonpost.com/entry/facebook-removed-fake-accounts_us_5afbeed1e4b06a3fb50c0458,facebooks community standards enforcement report shows company trying open executive said,2018-05-16,facebooks community standards enforcement report shows company trying open executive said,Neutral
560,568,TECH,"Tesla Autopilot Was Engaged During Horrific 60 MPH Crash, Driver Tells Police",Mary Papenfuss,https://www.huffingtonpost.com/entry/autopilot-was-engaged-in-tesla-crash_us_5afa3157e4b044dfffb52c73,tesla ceo elon musk tweeted amazing driver wasnt seriously injured,2018-05-15,tesla ceo elon musk tweeted amazing driver wasnt seriously injured,Positive
612,620,TECH,Facebook Suspends 200 Apps Amid Data Misuse Investigation,"Supantha Mukherjee, Reuters",https://www.huffingtonpost.com/entry/facebook-200-apps-suspended-data_us_5af99d6ee4b00d7e4c1cfd81,investigation launched response cambridge analytica scandal,2018-05-14,investigation launched response cambridge analytica scandal,Negative
663,672,TECH,Keyless Cars Have Killed More Than 2 Dozen People Since 2006: Report,Nina Golgowski,https://www.huffingtonpost.com/entry/carbon-monoxide-poisoning-keyless-cars_us_5af9a863e4b044dfffb4774d,drivers succumbed carbon monoxide poisoning forgetting turn vehicle garage,2018-05-14,drivers succumbed carbon monoxide poisoning forgetting turn vehicle garage,Negative
671,680,TECH,Possible Autopilot Use Probed After Tesla Crashes At 60 MPH,Mary Papenfuss,https://www.huffingtonpost.com/entry/tesla-with-autopilot-rams-truck_us_5af7a084e4b00d7e4c1b3cb8,known autopilot engaged cars driver suffered broken ankle,2018-05-13,known autopilot engaged cars driver suffered broken ankle,Negative


In [6]:
testDataset.toPandas()

,_c0,index,category,headline,authors,link,short_description,date,short_description_new,label
0,136,137,TECH,Facebook Accused Of Reading Texts And Accessin...,Sara Boboltz,https://www.huffingtonpost.com/entry/facebook-...,failed startup suggested facebook mounted desc...,2018-05-24,failed startup suggested facebook mounted desc...,Negative
1,137,138,TECH,Self-Driving Uber In Fatal Accident Had 6 Seco...,Ryan Grenoble,https://www.huffingtonpost.com/entry/self-driv...,ntsb published preliminary report incident thu...,2018-05-24,ntsb published preliminary report incident thu...,Neutral
2,270,272,TECH,Instagram Is Adding A 'Mute' Button For The Sa...,Sara Boboltz,https://www.huffingtonpost.com/entry/instagram...,know use,2018-05-22,use,Neutral
3,369,373,TECH,Cambridge Analytica Files For Chapter 7 Bankru...,None,https://www.huffingtonpost.com/entry/cambridge...,firm center facebook data hack decision suffer...,2018-05-18,firm center facebook data hack decision suffer...,Negative
4,493,499,TECH,Why Isn't There A Birth Control Emoji?,"Nicole Lee, Engadget",https://www.huffingtonpost.com/entry/why-isnt-...,women decided time,2018-05-16,women decided time,Neutral
5,512,519,TECH,Facebook Removed More Than 500 Million Fake Ac...,Willa Frej,https://www.huffingtonpost.com/entry/facebook-...,facebooks community standards enforcement repo...,2018-05-16,facebooks community standards enforcement repo...,Neutral
6,560,568,TECH,Tesla Autopilot Was Engaged During Horrific 60...,Mary Papenfuss,https://www.huffingtonpost.com/entry/autopilot...,tesla ceo elon musk tweeted amazing driver was...,2018-05-15,tesla ceo elon musk tweeted amazing driver was...,Positive
7,612,620,TECH,Facebook Suspends 200 Apps Amid Data Misuse In...,"Supantha Mukherjee, Reuters",https://www.huffingtonpost.com/entry/facebook-...,investigation launched response cambridge anal...,2018-05-14,investigation launched response cambridge anal...,Negative
8,663,672,TECH,Keyless Cars Have Killed More Than 2 Dozen Peo...,Nina Golgowski,https://www.huffingtonpost.com/entry/carbon-mo...,drivers succumbed carbon monoxide poisoning fo...,2018-05-14,drivers succumbed carbon monoxide poisoning fo...,Negative
9,671,680,TECH,Possible Autopilot Use Probed After Tesla Cras...,Mary Papenfuss,https://www.huffingtonpost.com/entry/tesla-wit...,known autopilot engaged cars driver suffered b...,2018-05-13,known autopilot engaged cars driver suffered b...,Negative


In [7]:
trainDataset.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2274 entries, 0 to 2273
Data columns (total 10 columns):
_c0 2274 non-null object
index 2274 non-null object
category 2274 non-null object
headline 2274 non-null object
authors 1878 non-null object
link 1980 non-null object
short_description 1834 non-null object
date 1722 non-null object
short_description_new 1679 non-null object
label 1687 non-null object
dtypes: object(10)
memory usage: 177.7+ KB

In [8]:
trainDataset = trainDataset.na.drop()

In [9]:
trainDataset= trainDataset.filter((trainDataset["label"]=="Positive") | (trainDataset["label"]=="Negative") | (trainDataset["label"]=="Neutral"))


In [10]:
trainDataset.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1282 entries, 0 to 1281
Data columns (total 10 columns):
_c0 1282 non-null object
index 1282 non-null object
category 1282 non-null object
headline 1282 non-null object
authors 1282 non-null object
link 1282 non-null object
short_description 1282 non-null object
date 1282 non-null object
short_description_new 1282 non-null object
label 1282 non-null object
dtypes: object(10)
memory usage: 100.2+ KB

In [11]:
trainDataset.toPandas()['label'].value_counts()

Out[13]: Neutral 544
Positive 438
Negative 300
Name: label, dtype: int64

In [12]:
trainDataset.show()

+------+------+--------+--------------------+--------------------+--------------------+--------------------+----------+---------------------+--------+
 _c0| index|category| headline| authors| link| short_description| date|short_description_new| label|
+------+------+--------+--------------------+--------------------+--------------------+--------------------+----------+---------------------+--------+
 59991| 62030| TECH|Report: White Hou...|Mark Hosenball an...|https://www.huffi...|administration re...|2016-04-07| administration re...| Neutral|
 46674| 47896| TECH|Samsung Galaxy No...|Ankur Banerjee, R...|https://www.huffi...|agency said owner...|2016-09-15| agency said owner...|Negative|
 15783| 16008| TECH|The 7 Problems Wi...|The School Of Lif...|https://www.huffi...|addicted phones r...|2017-09-21| addicted phones r...| Neutral|
 61052| 63129| TECH|Interactive Maps ...| Casey Williams|https://www.huffi...| city sound|2016-03-25| city sound| Neutral|
126804|148394| TECH|New iPhone To Be ...| Reuters, Reuters|https://www.huffi...|reuters apple wor...|2013-08-11| reuters apple wor...| Neutral|
 512| 519| TECH|Facebook Removed ...| Willa Frej|https://www.huffi...|facebooks communi...|2018-05-16| facebooks communi...| Neutral|
 10868| 11012| TECH|Elon Musk's Tweet...| Doha Madani|https://www.huffi...|years bot fast yo...|2017-11-26| years bot fast yo...|Positive|
 71939| 74400| TECH|Apple Pay Is Expa...| Anick Jesdanun, AP|https://www.huffi...|mobile payment se...|2015-11-17| mobile payment se...| Neutral|
175741|198203| TECH|Sony Xperia P, Xp...| Reuters, Reuters|https://www.huffi...|thats way envisio...|2012-02-26| envision sony mob...|Negative|
 60867| 62929| TECH|DOJ Says It Hacke...| Dan Levine, Reuters|https://www.huffi...|feds withdrew leg...|2016-03-28| feds withdrew leg...|Positive|
 50947| 52432| TECH|Nintendo Stock Ta...|Junko Fujita, Reu...|https://www.huffi...|app actually wont...|2016-07-25| app actually wont...| Neutral|
 82901| 85581| TECH|Samsung Just Unve...| Alexandra Ma|https://www.huffi...| hot damn thing|2015-07-15| hot damn thing|Negative|
121042|142467| TECH|Larry King And Ot...| Alexis Kleinman|https://www.huffi...|rainn wilson offi...|2013-10-13| rainn wilson offi...|Positive|
150188|172191| TECH|John McAfee Captu...| Dominique Mosbergen|https://www.huffi...|blog example mcaf...|2012-12-02| blog example mcaf...|Negative|
150998|173010| TECH|Fighting Hurrican...| Reuters, Reuters|https://www.huffi...|officials dealing...|2012-11-23| officials dealing...|Negative|
 71682| 74137| TECH|As Encryption Deb...| Alexander Howard|https://www.huffi...|sides arent exact...|2015-11-20| sides arent exact...|Negative|
 82788| 85466| TECH|'Angry Birds 2' I...| Damon Beres|https://www.huffi...| ready thy thumbs|2015-07-16| ready thy thumbs|Positive|
 52193| 53739| TECH|'Pokémon Go' Just...| Damon Beres|https://www.huffi...| meowth thats right|2016-07-11| meowth right| Neutral|
 59141| 61130| TECH|Steve Wozniak Jus...| Casey Williams|https://www.huffi...|thats think uber ...|2016-04-18| think uber nice t...|Positive|
 7804| 7900| TECH|YouTube's 'Open L...| David Moye|https://www.huffi...|youtube didnt add...|2018-01-10| youtube didnt add...|Negative|
+------+------+--------+--------------------+--------------------+--------------------+--------------------+----------+---------------------+--------+
only showing top 20 rows

In [13]:
testDataset = testDataset.na.drop()

In [14]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline

sparknlp.start()

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [15]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("short_description_new")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [16]:
trainDataset

Out[18]: DataFrame[_c0: string, index: string, category: string, headline: string, authors: string, link: string, short_description: string, date: string, short_description_new: string, label: string]

In [17]:
trainDataset.filter("short_description_new IS NULL").count()

Out[19]: 0

In [18]:

pipelineModel = pipeline.fit(trainDataset.filter("label IN ('Positive','Negative')"))


In [19]:
predictions =pipelineModel.transform(testDataset.filter("label IN ('Positive','Negative')"))
display(predictions)


_c0 index category headline authors link short_description date short_description_new label document sentence_embeddings class 136 137 TECH Facebook Accused Of Reading Texts And Accessing Microphones In Lawsuit Sara Boboltz https://www.huffingtonpost.com/entry/facebook-allegations-reading-texts-accessing-microphones_us_5b06c226e4b07c4ea105b154 failed startup suggested facebook mounted described mass surveillance 2018-05-24 failed startup suggested facebook mounted described mass surveillance Negative List(List(document, 0, 68, failed startup suggested facebook mounted described mass surveillance, Map(sentence -> 0), List())) List(List(sentence_embeddings, 0, 68, failed startup suggested facebook mounted described mass surveillance, Map(sentence -> 0, token -> failed startup suggested facebook mounted described mass surveillance, pieceId -> -1, isWordStart -> true), List(0.04750633, -0.06117647, -0.05010971, 0.049069945, -0.013978027, 0.055626247, 0.04314377, -0.021535957, -0.06360333, 0.043367084, -0.006797343, 0.040940996, -0.018873869, 0.031400293, 0.024331389, -0.052696917, -0.058806684, 0.040607523, 0.036785014, 0.027561039, -0.0299708, -0.057929445, 0.021772766, 0.06301688, 0.057784725, 0.004189002, -0.038612157, -0.025111277, 0.07954005, 0.007869317, 0.04345152, 0.017179413, -0.0010080824, -0.02910598, -0.0081647225, 0.0020596406, 0.014667981, 0.024727017, -0.02098624, -0.04614702, -0.05787225, -0.009620987, 0.030008556, 0.081380844, -0.016131377, 0.024508486, -0.06512189, -0.060460854, 0.053991947, -0.07094338, -0.034618475, 0.009664567, -0.024103632, 0.017104646, 0.016721405, 0.06779575, 0.059296463, -0.08174641, 0.07489735, -0.010986851, 0.031517424, -0.026753303, -0.05637422, 0.05437897, -0.008607002, -0.041770138, 0.06043466, 0.060552415, -0.0052546496, 0.03245716, -0.027388174, -4.016933E-4, -0.07482458, 0.07059212, 0.011793503, -0.033487648, 0.028993413, 0.05160861, 0.042087078, -0.059916113, 0.036432996, 0.0558008, -0.0025433367, 0.052427314, 0.033286452, 0.021798147, -0.034784883, -0.047742613, 0.03426118, -0.026607217, 0.05841711, 0.038799707, 0.0087179765, 0.049878974, -0.009339297, -0.08104689, 0.06270404, -0.062871575, -0.031994265, 0.037196007, -0.039012335, 0.009076562, -0.044909835, 2.9540897E-4, -0.045297872, 0.08436959, 0.024994677, -0.023706699, -0.0587263, 0.016912382, -0.08364917, -0.055480856, 0.04669218, -0.054673992, -0.031950153, -0.029494204, 0.01621632, -0.04691744, 0.082078144, -0.0021133837, -0.026602913, -0.06660055, -0.0549028, -0.030307526, 0.080069944, -0.0021221132, 0.01200285, 0.012406472, 0.0066306097, 0.07574956, -0.053754, 0.03648698, 0.03434122, -0.030503862, 0.04119543, -0.03870707, 0.067216784, 0.023002608, 0.0044606025, -0.052593447, -0.07646238, 0.04682531, -0.0037550821, -0.009331853, -0.016633313, -0.042112984, -0.029055497, -0.04713864, 0.06957031, 0.038131393, 0.07176328, -0.05139532, -0.019729605, 0.05108978, -0.029983755, 0.013469212, -0.008305303, -0.0288808, -0.01596415, 0.026201045, -0.041182093, 0.020162566, 0.013384731, 0.081110075, -0.023543928, 0.03843293, -0.038419817, 0.016453901, 0.03731119, -0.056310464, -0.05736777, -0.027817639, -0.049218163, 0.07416203, -0.00858524, 0.08075304, 0.083066024, 0.05798544, -0.0034496924, -0.065596804, 0.009532278, 0.022948848, -0.031930495, 0.011125021, -0.01093251, -0.034716886, -0.043948837, -0.008519098, 0.03178348, 0.0040651737, -0.015638327, 0.03150986, -0.009504692, 0.06558192, 0.035490386, -0.039519597, 0.03313488, 0.04510644, 0.08097926, 0.018694095, 0.054260712, 0.06568611, 0.050376903, 0.063340805, 0.060725145, -0.03750747, -0.026683403, -0.04687422, 0.048199948, 0.0671474, 0.029159132, -0.020197062, -0.013400283, -0.058964584, 0.081061706, 0.022186665, 0.0558196, -0.006538619, 0.03497753, -5.0687515E-5, 0.030738171, -0.039917096, -0.07889597, -0.06864039, -0.021081245, 0.061581008, -0.038955104, 0.03742321, -0.0075003835, -0.029151738, 0.07392706, -0.075389646, 0.054010913, 0.0012989892, -0.039510395, 0.029957063, -0.0

In [20]:
new_predictions = predictions.selectExpr("*", "class.result[0] as prediction")
display(new_predictions.select("label", "prediction"))

label,prediction
Negative,Negative
Positive,Positive
Negative,Negative
Negative,Negative
Negative,Negative
Positive,Positive
Positive,Positive
Negative,Negative
Negative,Negative
Positive,Positive


In [21]:
@F.udf(returnType="double")
def convert_label(label):
  if label == "Positive":
    return 1.0
  if label == "Negative":
    return -1.0
  return 0.0

new_new_predictions = new_predictions \
                    .withColumn("label_double", convert_label("label")) \
                    .withColumn("prediction_double", convert_label("prediction"))
display(new_new_predictions.select("label_double", "prediction_double"))

label_double,prediction_double
-1.0,-1.0
1.0,1.0
-1.0,-1.0
-1.0,-1.0
-1.0,-1.0
1.0,1.0
1.0,1.0
-1.0,-1.0
-1.0,-1.0
1.0,1.0


In [22]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_double", predictionCol="prediction_double",                                                  metricName="accuracy")
evaluator.evaluate(new_new_predictions)

Out[24]: 0.839622641509434

In [23]:
predictions.printSchema()

root
-- _c0: string (nullable = true)
-- index: string (nullable = true)
-- category: string (nullable = true)
-- headline: string (nullable = true)
-- authors: string (nullable = true)
-- link: string (nullable = true)
-- short_description: string (nullable = true)
-- date: string (nullable = true)
-- short_description_new: string (nullable = true)
-- label: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence_embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- class: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [24]:
predictions = predictions.selectExpr("*", "class.result[0] as prediction")
display(predictions)

_c0 index category headline authors link short_description date short_description_new label document sentence_embeddings class prediction 136 137 TECH Facebook Accused Of Reading Texts And Accessing Microphones In Lawsuit Sara Boboltz https://www.huffingtonpost.com/entry/facebook-allegations-reading-texts-accessing-microphones_us_5b06c226e4b07c4ea105b154 failed startup suggested facebook mounted described mass surveillance 2018-05-24 failed startup suggested facebook mounted described mass surveillance Negative List(List(document, 0, 68, failed startup suggested facebook mounted described mass surveillance, Map(sentence -> 0), List())) List(List(sentence_embeddings, 0, 68, failed startup suggested facebook mounted described mass surveillance, Map(sentence -> 0, token -> failed startup suggested facebook mounted described mass surveillance, pieceId -> -1, isWordStart -> true), List(0.04750633, -0.06117647, -0.05010971, 0.049069945, -0.013978027, 0.055626247, 0.04314377, -0.021535957, -0.06360333, 0.043367084, -0.006797343, 0.040940996, -0.018873869, 0.031400293, 0.024331389, -0.052696917, -0.058806684, 0.040607523, 0.036785014, 0.027561039, -0.0299708, -0.057929445, 0.021772766, 0.06301688, 0.057784725, 0.004189002, -0.038612157, -0.025111277, 0.07954005, 0.007869317, 0.04345152, 0.017179413, -0.0010080824, -0.02910598, -0.0081647225, 0.0020596406, 0.014667981, 0.024727017, -0.02098624, -0.04614702, -0.05787225, -0.009620987, 0.030008556, 0.081380844, -0.016131377, 0.024508486, -0.06512189, -0.060460854, 0.053991947, -0.07094338, -0.034618475, 0.009664567, -0.024103632, 0.017104646, 0.016721405, 0.06779575, 0.059296463, -0.08174641, 0.07489735, -0.010986851, 0.031517424, -0.026753303, -0.05637422, 0.05437897, -0.008607002, -0.041770138, 0.06043466, 0.060552415, -0.0052546496, 0.03245716, -0.027388174, -4.016933E-4, -0.07482458, 0.07059212, 0.011793503, -0.033487648, 0.028993413, 0.05160861, 0.042087078, -0.059916113, 0.036432996, 0.0558008, -0.0025433367, 0.052427314, 0.033286452, 0.021798147, -0.034784883, -0.047742613, 0.03426118, -0.026607217, 0.05841711, 0.038799707, 0.0087179765, 0.049878974, -0.009339297, -0.08104689, 0.06270404, -0.062871575, -0.031994265, 0.037196007, -0.039012335, 0.009076562, -0.044909835, 2.9540897E-4, -0.045297872, 0.08436959, 0.024994677, -0.023706699, -0.0587263, 0.016912382, -0.08364917, -0.055480856, 0.04669218, -0.054673992, -0.031950153, -0.029494204, 0.01621632, -0.04691744, 0.082078144, -0.0021133837, -0.026602913, -0.06660055, -0.0549028, -0.030307526, 0.080069944, -0.0021221132, 0.01200285, 0.012406472, 0.0066306097, 0.07574956, -0.053754, 0.03648698, 0.03434122, -0.030503862, 0.04119543, -0.03870707, 0.067216784, 0.023002608, 0.0044606025, -0.052593447, -0.07646238, 0.04682531, -0.0037550821, -0.009331853, -0.016633313, -0.042112984, -0.029055497, -0.04713864, 0.06957031, 0.038131393, 0.07176328, -0.05139532, -0.019729605, 0.05108978, -0.029983755, 0.013469212, -0.008305303, -0.0288808, -0.01596415, 0.026201045, -0.041182093, 0.020162566, 0.013384731, 0.081110075, -0.023543928, 0.03843293, -0.038419817, 0.016453901, 0.03731119, -0.056310464, -0.05736777, -0.027817639, -0.049218163, 0.07416203, -0.00858524, 0.08075304, 0.083066024, 0.05798544, -0.0034496924, -0.065596804, 0.009532278, 0.022948848, -0.031930495, 0.011125021, -0.01093251, -0.034716886, -0.043948837, -0.008519098, 0.03178348, 0.0040651737, -0.015638327, 0.03150986, -0.009504692, 0.06558192, 0.035490386, -0.039519597, 0.03313488, 0.04510644, 0.08097926, 0.018694095, 0.054260712, 0.06568611, 0.050376903, 0.063340805, 0.060725145, -0.03750747, -0.026683403, -0.04687422, 0.048199948, 0.0671474, 0.029159132, -0.020197062, -0.013400283, -0.058964584, 0.081061706, 0.022186665, 0.0558196, -0.006538619, 0.03497753, -5.0687515E-5, 0.030738171, -0.039917096, -0.07889597, -0.06864039, -0.021081245, 0.061581008, -0.038955104, 0.03742321, -0.0075003835, -0.029151738, 0.07392706, -0.075389646, 0.054010913, 0.0012989892, -0.039510395, 0.0299

In [25]:
# pipelineModel.save("/mnt/mnt_s3/sentimentdl_pipeline_2")

In [26]:
# loadedPipeline = PipelineModel.load("/mnt/mnt_s3/sentimentdl_pipeline_2")

In [27]:
# import pandas as pd
# pdf = pd.DataFrame({"text": [
#   "This is so great! How are you", 
#   "No way, never again!", 
#   "Yes, bring on more!"
# ]})
# df = spark.createDataFrame(pdf)

# results = pipelineModel.transform(df)
# display(
#   results
# )

In [28]:
# display(
#   results.selectExpr("text", "class.result")
# )

In [29]:
# analyzer.polarity_scores("This is so great! How are you")

In [30]:
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# analyzer = SentimentIntensityAnalyzer()

# @F.udf(returnType="struct<neg:double,neu:double,pos:double,compound:double>")
# def getVADER(text):
# 	return analyzer.polarity_scores(text)

# new_results = results.withColumn("vaderSentiment", getVADER("text"))
# display(new_results)